In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import datetime as dt
import time

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

import keras

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

# from keras.applications import vgg16, resnet50, mobilenet
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50
# from keras.applications import xception
# from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

C:\Users\Andromeda\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [137]:
import os
import shutil
from tqdm import tqdm

# move file in accordance to percent
def move(from_path, to_path, percent):
    from_small = False
    from_files = os.listdir(from_path)
    to_files = os.listdir(to_path)
    
    from_files_len = len(from_files)
    to_files_len = len(to_files)
    
    total_files_num = from_files_len + to_files_len
    
    max_num = (total_files_num)*(percent/100)
    max_num = max_num-1
    
    if from_files_len<to_files_len:
        from_path, to_path = to_path, from_path
        from_files = os.listdir(from_path)
        to_files = os.listdir(to_path)
        
    
    i=len(to_files)-1
    
    for name in tqdm(from_files):
        from_small = True
        if i>=max_num:
            break
            
        from_file = os.path.join(from_path, name)
        to_file = os.path.join(to_path, name)
        shutil.move(from_file, to_file)
        i+=1

    if from_small:
        from_path, to_path = to_path, from_path
    
    from_files = os.listdir(from_path)
    to_files = os.listdir(to_path)
    
    from_files_len = len(from_files)
    to_files_len = len(to_files)
    
    print("Percentile: %d:\nFrom(%s):%s\nTo(%s):%s"% (percent, from_path, from_files_len, to_path, to_files_len))
    
    return from_files_len, to_files_len

In [138]:
test_norm = r'data\test\NORMAL'
test_pneu = r'data\test\PNEUMONIA'

val_norm = r'data\val\NORMAL'
val_pneu = r'data\val\PNEUMONIA'

test_norm_files = os.listdir(test_norm)
test_pneu_files = os.listdir(test_pneu)

print('Data load initiated')

move(test_norm, val_norm, 50)
move(test_pneu, val_pneu, 50)


Data load initiated


  0%|          | 0/121 [00:00<?, ?it/s]


Percentile: 50:
From(data\val\NORMAL):121
To(data\test\NORMAL):121


  0%|          | 0/199 [00:00<?, ?it/s]


Percentile: 50:
From(data\val\PNEUMONIA):199
To(data\test\PNEUMONIA):199


(199, 199)

In [139]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [140]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [141]:
# #Load the VGG model
# vgg_model = vgg16.VGG16(weights='imagenet')
 
# #Load the Inception_V3 model
# inception_model = inception_v3.InceptionV3(weights='imagenet')
 
# #Load the ResNet50 model
# resnet_model = resnet50.ResNet50(weights='imagenet')
 
# #Load the MobileNet model
# mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [142]:
training_path = 'data\\train'
validation_path = 'data\\val'
testing_path = 'data\\test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

NameError: name 'norm' is not defined

In [144]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [145]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)


In [146]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [147]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True


0 input_3
1 conv2d_189
2 batch_normalization_189
3 activation_189
4 conv2d_190
5 batch_normalization_190
6 activation_190
7 conv2d_191
8 batch_normalization_191
9 activation_191
10 max_pooling2d_9
11 conv2d_192
12 batch_normalization_192
13 activation_192
14 conv2d_193
15 batch_normalization_193
16 activation_193
17 max_pooling2d_10
18 conv2d_197
19 batch_normalization_197
20 activation_197
21 conv2d_195
22 conv2d_198
23 batch_normalization_195
24 batch_normalization_198
25 activation_195
26 activation_198
27 average_pooling2d_19
28 conv2d_194
29 conv2d_196
30 conv2d_199
31 conv2d_200
32 batch_normalization_194
33 batch_normalization_196
34 batch_normalization_199
35 batch_normalization_200
36 activation_194
37 activation_196
38 activation_199
39 activation_200
40 mixed0
41 conv2d_204
42 batch_normalization_204
43 activation_204
44 conv2d_202
45 conv2d_205
46 batch_normalization_202
47 batch_normalization_205
48 activation_202
49 activation_205
50 average_pooling2d_20
51 conv2d_201
52 

In [148]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

sgd = optimizers.Adam()
# sgd = optimizers.SGD()
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = optimizer=SGD(lr=0.0001, momentum=0.9)

model.compile(sgd, loss='categorical_crossentropy', metrics=["accuracy"])

# model.compile(loss='mean_squared_error', optimizer='sgd')

In [149]:
model_dir = 'outputs/models/'
log_file = "outputs/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [150]:
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [151]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

In [ ]:
# callbacks_list = [checkpoint, tensorboard, early_stopping]
callbacks_list = [checkpoint, tensorboard]

In [ ]:
def clear_dir(directory):
    if os.path.exists(directory):
        try:
            shutil.rmtree(directory)
            os.mkdir(directory)
        except:
            print("error:", directory)
    else:
        try:
            os.mkdir(directory)
        except:
            print("error create:", directory)

In [ ]:
clear_dir(model_dir)
clear_dir(log_file)

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [79]:
filepath = 'outputs/models/weights-improvement-01-0.92.hdf5'
model = keras.models.load_model(filepath)

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

In [133]:
# filepath = 'outputs/models/weights-improvement-01-0.65.hdf5'
# filepath = 'outputs/models/weights-improvement-02-0.66.hdf5'
# filepath = 'outputs/models/weights-improvement-03-0.88.hdf5'
filepath = 'outputs/models/weights-improvement-04-0.75.hdf5'


# filepath = 'outputs/models/weights-improvement-01-0.92.hdf5'
# filepath = 'outputs/models/weights-improvement-02-0.83.hdf5'

model = keras.models.load_model(filepath)

In [134]:
result = model.evaluate_generator(generator=test_generator, verbose=1)

10/10 [==============================] - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 53s - ETA: 39 - ETA: 25 - ETA: 2: - 1569s 157s/step


In [135]:
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

Accuracy: 75.31%
Loss: 0.47


In [97]:
preds = model.predict_generator(test_generator, verbose=1)

10/10 [==============================] - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 42 - ETA: 28 - ETA: 14 - 142s 14s/step


In [98]:
categorical_accuracy = keras.metrics.categorical_accuracy(test_generator.classes, preds)

In [101]:
categorical_accuracy

<tf.Tensor 'Cast_2:0' shape=(320,) dtype=float32>

In [ ]:
'''
filepath = 'outputs/models/weights-improvement-01-0.66.hdf5'


filepath = 'outputs/models/weights-improvement-02-0.68.hdf5'


filepath = 'outputs/models/weights-improvement-03-0.88.hdf5'


filepath = 'outputs/models/weights-improvement-04-0.75.hdf5'
Accuracy: 75.31%
Loss: 0.47


filepath = 'outputs/models/weights-improvement-01-0.92.hdf5'
Accuracy: 80.94%
Loss: 0.47
'''
# tensorboard --logdir=logs

In [ ]:
def get_class_values(gen):
    vals = []
    for i in gen:
        vals.append(gen[i])
    return vals

def get_classes(gen):
    rev_gen = {}
    for i in gen:
        rev_gen[gen[i]] = i
    return rev_gen

def find_max(preds_i):
    num = -1
    index = -1
    for i in range(len(preds_i)):
        if preds_i[i]>num:
            num = preds_i[i]
            index = i
    return index  

def get_label(cls, val):
    return cls[val]

In [ ]:
vals = get_class_values(test_generator.class_indices)
cls = get_classes(test_generator.class_indices)

In [ ]:
total = 0
acc = 0
direc = "data\\pre\\test"
files = test_generator.filenames
for i in range(len(preds)):
    cls_index = find_max(preds[i])
    pred_label = cls[cls_index]
    org_label = cls[test_generator.classes[i]]
    file_name = os.path.join(direc, files[i])
    if org_label==pred_label:
        acc +=1
    if total%10==0:
        result = "Wrong!!!\n"
        if org_label==pred_label:
            result = "Right!!!\n"
        title_text = "Original: "+ org_label+"\nPredicted: "+ pred_label+"\nFile: "+ file_name
        title_text=result+ title_text
        
        img = cv2.imread(file_name, 3)
        if org_label==pred_label:
            plt.title(title_text, color='blue')
        else:
            plt.title(title_text, color='red')
        plt.imshow(img)
        plt.show()
    
    total+=1
accuracy = (acc/total)*100
print("Accuracy: ", accuracy)